In [0]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.utils import np_utils
import requests
import re

In [0]:
sonnet_url = "http://www.gutenberg.org/cache/epub/1041/pg1041.txt"
sonnets = requests.get(sonnet_url).text
text = sonnets.lower()

In [11]:
from google.colab import files
uploaded = files.upload()

Saving all.csv to all.csv


In [0]:
poems = pd.read_csv("all.csv", encoding="utf8")

In [18]:
poem_content[:5000]

'Let the bird of loudest lay\r\nOn the sole Arabian tree\r\nHerald sad and trumpet be,\r\nTo whose sound chaste wings obey.\r\n\r\nBut thou shrieking harbinger,\r\nFoul precurrer of the fiend,\r\nAugur of the fever\'s end,\r\nTo this troop come thou not near.\r\n\r\nFrom this session interdict\r\nEvery fowl of tyrant wing,\r\nSave the eagle, feather\'d king;\r\nKeep the obsequy so strict.\r\n\r\nLet the priest in surplice white,\r\nThat defunctive music can,\r\nBe the death-divining swan,\r\nLest the requiem lack his right.\r\n\r\nAnd thou treble-dated crow,\r\nThat thy sable gender mak\'st\r\nWith the breath thou giv\'st and tak\'st,\r\n\'Mongst our mourners shalt thou go.\r\n\r\nHere the anthem doth commence:\r\nLove and constancy is dead;\r\nPhoenix and the Turtle fled\r\nIn a mutual flame from hence.\r\n\r\nSo they lov\'d, as love in twain\r\nHad the essence but in one;\r\nTwo distincts, division none:\r\nNumber there in love was slain.\r\n\r\nHearts remote, yet not asunder;\r\nDis

In [0]:
poem_clean = re.sub(r'[^\x00-\x7F]+',' ', poem_content)

In [36]:
len(poem_clean)

568917

In [0]:
characters = sorted(list(set(poem_clean)))
n_to_char = {n:char for n, char in enumerate(characters)}
char_to_n = {char:n for n, char in enumerate(characters)}

In [0]:
X = []
Y = []
length = len(poem_clean)
seq_length = 100
for i in range(0, length-seq_length, 1):
   sequence = poem_clean[i:i + seq_length]
   label = poem_clean[i + seq_length]
   X.append([char_to_n[char] for char in sequence])
   Y.append(char_to_n[label])

In [0]:
X_modified = np.reshape(X, (len(X), seq_length, 1))
X_modified = X_modified / float(len(characters))
Y_modified = np_utils.to_categorical(Y)

In [0]:
model = Sequential()
model.add(LSTM(400, input_shape=(X_modified.shape[1], X_modified.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(400))
model.add(Dropout(0.2))
model.add(Dense(Y_modified.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [35]:
model.fit(X_modified, Y_modified, epochs=1, batch_size=100)

Epoch 1/1
  1400/568817 [..............................] - ETA: 5:33:02 - loss: 3.6130

KeyboardInterrupt: ignored

In [9]:
string_mapped = X[97]
print(len(string_mapped))
full_string = [n_to_char[value] for value in string_mapped]
# generating characters
for i in range(400):
    x = np.reshape(string_mapped,(1,len(string_mapped), 1))
    x = x / float(len(characters))

    pred_index = np.argmax(model.predict(x, verbose=0))
    seq = [n_to_char[value] for value in string_mapped]
    full_string.append(n_to_char[pred_index])

    string_mapped.append(pred_index)
    string_mapped = string_mapped[1:len(string_mapped)]

100


In [10]:

#combining text
txt=""
for char in full_string:
    txt = txt+char
txt

'r the use of anyone anywhere at no cost and with\r\nalmost no restrictions whatsoever.  you may copy io the the the the thee \r  the the the the the the the she thee \r\n    xxi \n\r\n  th the the the the the the the the shee \r  the the the the the the the she thee \r  the the the the the the the she thee \r  the the the the the the the she thee \r  the the the the the the the she thee \r  the the the the the the the she thee \r  the the the the the the the she thee \r  the the the the the the the she thee \r '

In [0]:
full_string